In [5]:
import pyterrier as pt
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from ir_measures import nDCG

In [6]:
ensure_pyterrier_is_loaded()
ds_id = "radboud-validation-20251114-training"
dataset = pt.datasets.get_dataset(f"irds:ir-lab-wise-2025/{ds_id}")
topics = dataset.get_topics("title")
qrels = dataset.get_qrels()

#### Does the choice of relevance feedback method (Bo1 vs. RM3) in a DPH-based Retriever → Rewriter → Retriever pipeline significantly affect retrieval effectiveness measured by nDCG@10 on the radboud-validation-20251114-training dataset?

The Rewriter component performs query expansion exclusively via pseudo-relevance feedback. No additional query reformulation or term weighting beyond Bo1 or RM3 is applied.

In [ ]:
# ows_bm_25 = pt.Artifact.from_url("tira:radboud-validation-20251114-training/ows/pyterrier-BM25-on-title")
dph_bo1_dph = pt.Artifact.from_url("tira:radboud-validation-20251114-training/ks-golden-retrievals/pyterrier-on-default_text-with-DPH-Bo1-DPH")
dph_rm3_dph = pt.Artifact.from_url("tira:radboud-validation-20251114-training/ks-golden-retrievals/pyterrier-on-default_text-with-DPH-RM3-DPH-no-reformulation")

Does the choice of relevance feedback method (Bo1 vs. RM3) in a DPH-based Retriever → Rewriter → Retriever pipeline significantly affect retrieval effectiveness measured by nDCG@10 on the radboud-validation-20251114-training dataset?

The Rewriter component performs query expansion exclusively via pseudo-relevance feedback. No additional query reformulation or term weighting beyond Bo1 or RM3 is applied.

H₁:
Given query expansion via pseudo-relevance feedback using the top 3 retrieved documents and 10 expansion terms, there is a statistically significant difference in nDCG@10 on radboud-validation-20251114-training between (i) the DPH-based retrieval pipeline described in Section three with Bo1 query expansion and (ii) the same pipeline with RM3 query expansion. (α=0.05) 

H₀₁:
Given query expansion via pseudo-relevance feedback using the top 3 retrieved documents and 10 expansion terms, there is no statistically significant difference in nDCG@10 on radboud-validation-20251114-training between (i) the DPH-based retrieval pipeline described in Section three with Bo1 query expansion and (ii) the same pipeline with RM3 query expansion. (α=0.05) 

H₂:
Given query expansion via pseudo-relevance feedback using the top 3 retrieved documents and 10 expansion terms, there is a statistically significant improvement in nDCG@10 on radboud-validation-20251114-training between (i) the DPH-based retrieval pipeline described in Section three with Bo1 query expansion and (ii) the same pipeline with RM3 query expansion. (α=0.05)

H₀₂:
Given query expansion via pseudo-relevance feedback using the top 3 retrieved documents and 10 expansion terms, there is no statistically significant improvement in nDCG@10 on radboud-validation-20251114-training between (i) the DPH-based retrieval pipeline described in Section three with Bo1 query expansion and (ii) the same pipeline with RM3 query expansion. (α=0.05)

In [9]:
pt.Experiment(
    [dph_bo1_dph,dph_rm3_dph], # weighting techniques
    topics,
    qrels,
    ["ndcg_cut.10"],  # measure
    names=["DPH-Bo1-DPH", "DPH-RM3-DPH"],
    baseline=1, # ID of baseline
    test="t", # test to use; here: Student's t-test
    correction="bonferroni" # correction for multiple testing
)

,name,ndcg_cut.10,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value,ndcg_cut.10 reject,ndcg_cut.10 p-value corrected
0,DPH-Bo1-DPH,0.494674,17.0,8.0,0.282978,False,0.282978
1,DPH-RM3-DPH,0.474432,NaN,NaN,NaN,False,NaN


Though, Bo1 performs better than RM3. The p-value of 0.283 is significantly larger than 0.05. This means the observed difference is likely due to random variation across topics. We failed to reject our H₀₁ and H₀₂, which means we can't prove our H₁ and H₂.

Given the parameters of 3 retrieved documents and 10 expansion terms for Bo1 and RM3, both pseudo-relevance feedback-based transformer models behave similarly. Experimenting with more feedback documents and more expansion terms in the future could be meaningful.